In [2]:
from utils import gini, layer2, utils

import pandas as pd
import numpy as np

In [2]:
train, test = utils.load_data()

Successfully loaded train and test data


In [3]:
train = pd.read_csv('data/pca100_train.csv')
test = pd.read_csv('data/pca100_test.csv')

In [11]:
train = train.drop(['id','target'],axis=1)
test = test.drop(['id','target'], axis=1)

ModuleNotFoundError: No module named 'tensorflow'

In [90]:
X = layer2.get_meta_features('train')
X_test = layer2.get_meta_features('test')
y = pd.read_csv('data/train.csv').target
test_id = pd.read_csv('data/test.csv').id

In [91]:
for i in X.columns:
    print(i, gini.gini_normalized(y,X[i]))

adaboost 0.251812863624
lgb_with_dummies 0.284280083783
randomforest_5 0.24840956849
xgb_depth_10 0.282185081487
xgb_fillmed_123 0.285751516823


In [93]:
X.corr()

,adaboost,lgb_with_dummies,randomforest_5,xgb_depth_10,xgb_fillmed_123
adaboost,1.000000,0.820039,0.921247,0.783405,0.816613
lgb_with_dummies,0.820039,1.000000,0.830666,0.958740,0.981260
randomforest_5,0.921247,0.830666,1.000000,0.798696,0.826972
xgb_depth_10,0.783405,0.958740,0.798696,1.000000,0.968846
xgb_fillmed_123,0.816613,0.981260,0.826972,0.968846,1.000000


In [ ]:
n_round = 1000
score = {}
n_folds = 3
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=88)

for lap in range(n_round):
    score_lap = 0
    for train_index, val_index in skf.split(X, y):
        X_tr, X_vl = X.iloc[train_index,:], X.iloc[val_index,:]
        y_tr, y_vl = y[train_index], y[val_index]

        a = np.random.random()
        b = (1-a)*np.random.random()
        c = (1-a-b)

        
        pred = a*X_vl.xgb_depth_10+b*X_vl.xgb_fillmed_123+c*X_vl.lgb_with_dummies
        score_lap += gini.gini_normalized(y_vl, pred)/n_folds
    score['{}*X_test.xgb_depth_10+{}*X_test.xgb_fillmed_123+{}*X_test.lgb_with_dummies'.format(a,b,c)] = score_lap
    print('Round{} Score:{}'.format(lap+1, score_lap))

Round1 Score:0.28429397182965027
Round2 Score:0.28506214828326004
Round3 Score:0.28345568792588866
Round4 Score:0.2846482864966525
Round5 Score:0.285146724545595
Round6 Score:0.284669310349059
Round7 Score:0.28517823379018353
Round8 Score:0.28369745490228626
Round9 Score:0.28590225576638967
Round10 Score:0.2852297573561588
Round11 Score:0.2847894213481065
Round12 Score:0.2846506409724228
Round13 Score:0.2847912170155743
Round14 Score:0.2850377157843066
Round15 Score:0.2851296801045684
Round16 Score:0.28434873733163174
Round17 Score:0.28564019660530976
Round18 Score:0.2849096630757373
Round19 Score:0.28461826662245815
Round20 Score:0.2850125311278913
Round21 Score:0.28252635675450727
Round22 Score:0.2845386425209186
Round23 Score:0.28502073893882457
Round24 Score:0.2849801757910155
Round25 Score:0.2840995362293481
Round26 Score:0.2847684393937791
Round27 Score:0.2851058984547116
Round28 Score:0.28366339618250375
Round29 Score:0.28535559487190315
Round30 Score:0.2855095073159793
Round31 

Round243 Score:0.28425594650363367
Round244 Score:0.28494000288670357
Round245 Score:0.2846383522910165
Round246 Score:0.2849818907152305
Round247 Score:0.2841037415229996
Round248 Score:0.2857991397832554
Round249 Score:0.28439105380162316
Round250 Score:0.2846980404730185
Round251 Score:0.28398794411831946
Round252 Score:0.2858711199470584
Round253 Score:0.28445663890445866
Round254 Score:0.2856692879478677
Round255 Score:0.2846675590778148
Round256 Score:0.28533308321365675
Round257 Score:0.28561693715691283
Round258 Score:0.2845584314063405
Round259 Score:0.28479129083918553
Round260 Score:0.2848284563967887
Round261 Score:0.2846546038502617
Round262 Score:0.2853354340642416
Round263 Score:0.2844651246641091
Round264 Score:0.28446195172847794
Round265 Score:0.2849805121197812
Round266 Score:0.2845937942580123
Round267 Score:0.2851634390633688
Round268 Score:0.2850193876209382
Round269 Score:0.28576247066283794
Round270 Score:0.2857607310297669
Round271 Score:0.2857908716239563
Roun

In [82]:
score = pd.DataFrame.from_dict(score, orient='index')

In [83]:
score = score.sort_values(0, ascending=False)

In [85]:
pred = 0.15433875432023858*X_test.xgb_depth_10+0.6781272709781468*X_test.xgb_fillmed_123+0.008706714397084776*X_test.xgb_fillmed+0.1588272603045298*X_test.xgb

In [87]:
sub = pd.DataFrame({'id':test_id, 'target':pred})

In [89]:
sub.to_csv('../sub/sub46.csv', index=False)

In [ ]:
xgb_5